# 4 ways to print in GAlgebra

In [1]:
import sys
import subprocess
from IPython.core.display import display_pretty
from sympy import *
from galgebra.ga import Ga
from galgebra.printer import Eprint, Format, xpdf

In [2]:
cga3d = Ga(r'e_1 e_2 e_3 e e_{0}',g='1 0 0 0 0,0 1 0 0 0,0 0 1 0 0,0 0 0 0 -1,0 0 0 -1 0')

## Printing in plain text

In [3]:
print(cga3d.I())

e_1^e_2^e_3^e^e_{0}


##  Enhanced Console Printing

If called `Eprint()` upfront, `print()` will do Enhanced Console Printing(colored printing with ANSI escape sequences):

- code: https://github.com/pygae/galgebra/blob/master/examples/Terminal/terminal_check.py
- output: https://nbviewer.jupyter.org/github/pygae/galgebra/blob/master/examples/ipython/Terminal.ipynb

In [4]:
# Eprint modifies global state, start a new python process to avoid it
# affecting the rest of this notebook.
code = """
from galgebra.ga import Ga
from galgebra.printer import Eprint
Eprint()
cga3d = Ga(r'e_1 e_2 e_3 e e_{0}',g='1 0 0 0 0,0 1 0 0 0,0 0 1 0 0,0 0 0 0 -1,0 0 0 -1 0')
print(cga3d.I())

g3 = Ga('e*1|2|3')
print(g3.g)
"""
result = subprocess.check_output([sys.executable, '-c', code], universal_newlines=True)
display_pretty(result, raw=True)

e_1^e_2^e_3^e^e_{0}
Matrix([
[(e_1.e_1), (e_1.e_2), (e_1.e_3)],
[(e_1.e_2), (e_2.e_2), (e_2.e_3)],
[(e_1.e_3), (e_2.e_3), (e_3.e_3)]])


## MathJax printing in Jupyter Notebook

In a Jupyter Notebook upfront, without calling `print()`, it will do LaTeX printing with MathJax for Geometric Algebra expressions.

- code & output: https://nbviewer.jupyter.org/github/pygae/galgebra/blob/master/examples/ipython/colored_christoffel_symbols.ipynb

In [5]:
cga3d.I()

e_1^e_2^e_3^e^e_{0}

In versions of `sympy` from 1.4 onwards, the raw sympy expressions also print in the output:

In [6]:
from sympy import Basic
if not hasattr(Basic, '_repr_latex_'):  # sympy < 1.4
    def _repr_latex_(self):
        from sympy.printing.latex import latex
        s = latex(self, mode='plain')
        return "$\\displaystyle %s$" % s
    Basic._repr_latex_ = _repr_latex_

In [7]:
cga3d.I().obj

e_1^e_2^e_3^e^e_{0}

Calling `sympy.init_printing` enables printing of containers of objects too:

In [8]:
import sympy; sympy.init_printing(use_latex="mathjax")

In [9]:
g3 = Ga('e*1|2|3')
(g3.basis, g3.g)

⎛              ⎡(e₁⋅e₁)  (e₁⋅e₂)  (e₁⋅e₃)⎤⎞
⎜              ⎢                         ⎥⎟
⎜[e₁, e₂, e₃], ⎢(e₁⋅e₂)  (e₂⋅e₂)  (e₂⋅e₃)⎥⎟
⎜              ⎢                         ⎥⎟
⎝              ⎣(e₁⋅e₃)  (e₂⋅e₃)  (e₃⋅e₃)⎦⎠

In [10]:
o3d_no_wedge = Ga('e*1|2|3', g=[1, 1, 1], norm=True, wedge=False)
o3d_no_wedge.blades

((1,), (e₁, e₂, e₃), (e₁₂, e₁₃, e₂₃), (e₁₂₃,))

## LaTeX printing

If called `Format()` upfront and `xpdf()` eventually, `print()` will do LaTeX printing (internally, the standard output will be redirected to a buffer for later consumption).

- code: https://github.com/pygae/galgebra/blob/master/examples/LaTeX/colored_christoffel_symbols.py
- output: https://nbviewer.jupyter.org/github/pygae/galgebra/blob/master/examples/ipython/LaTeX.ipynb

In [11]:
Format()

In [12]:
print(cga3d.I())

In [13]:
xpdf(filename='test_latex_output.tex', paper=(9, 10), pdfprog=None)

In [14]:
with open('test_latex_output.tex') as f:
    data = f.read()
display_pretty(data, raw=True)


\documentclass[10pt,fleqn]{report}
\usepackage[vcentering]{geometry}
\geometry{papersize={9in,10in},total={8in,9in}}

\pagestyle{empty}
\usepackage[latin1]{inputenc}
\usepackage{amsmath}
\usepackage{amsfonts}
\usepackage{amssymb}
\usepackage{amsbsy}
\usepackage{tensor}
\usepackage{listings}
\usepackage{color}
\usepackage{xcolor}
\usepackage{bm}
\usepackage{breqn}
\definecolor{gray}{rgb}{0.95,0.95,0.95}
\setlength{\parindent}{0pt}
\DeclareMathOperator{\Tr}{Tr}
\DeclareMathOperator{\Adj}{Adj}
\newcommand{\bfrac}[2]{\displaystyle\frac{#1}{#2}}
\newcommand{\lp}{\left (}
\newcommand{\rp}{\right )}
\newcommand{\paren}[1]{\lp {#1} \rp}
\newcommand{\half}{\frac{1}{2}}
\newcommand{\llt}{\left <}
\newcommand{\rgt}{\right >}
\newcommand{\abs}[1]{\left |{#1}\right | }
\newcommand{\pdiff}[2]{\bfrac{\partial {#1}}{\partial {#2}}}
\newcommand{\lbrc}{\left \{}
\newcommand{\rbrc}{\right \}}
\newcommand{\W}{\wedge}
\newcommand{\prm}[1]{{#1}'}
\newcommand{\ddt}[1]{\bfrac{d{#1}}{dt}}
\newcommand{\R}{\dag

## Reproduce Some bugs

In [15]:
Format()
xyz_coords = x, y, z = symbols('x y z', real=True)
o3d, ex, ey, ez = Ga.build('e', g=[1, 1, 1], coords=xyz_coords, norm=True)

In [16]:
o3d.blades

((1,), (eₓ, e_y, e_z), (eₓ^e_y, eₓ^e_z, e_y^e_z), (eₓ^e_y^e_z,))

In [17]:
def build_ga_from_solution(solution, norm=False):
    [metric, coordinate_set, _index_config, cosmological_constant] = solution
    return Ga('e', g=metric, coords=coordinate_set, norm=norm)

g4coords = u, x, y, z = symbols("u x y z")
g = [
    [0, 0, -exp(-z), 0],
    [0, S.Half * u ** 2 * exp(4 * z), 0, 0],
    [-exp(-z), 0, 12 * exp(-2 * z), u * exp(-z)],
    [0, 0, u * exp(-z), S.Half * u ** 2],
]
g4 = build_ga_from_solution([g, g4coords, None, 0])

In [18]:
# NBVAL_IGNORE_OUTPUT
g4.mv()

(e_u, e_x, e_y, e_z)

In [19]:
# NBVAL_IGNORE_OUTPUT
g4.mvr()

(-10*e_u - exp(z)*e_y + 2*e_z/u, 2*exp(-4*z)*e_x/u**2, -exp(z)*e_u, 2*e_u/u + 2*e_z/u**2)

In [20]:
g4.blades

((1,), (eᵤ, eₓ, e_y, e_z), (eᵤ^eₓ, eᵤ^e_y, eᵤ^e_z, eₓ^e_y, eₓ^e_z, e_y^e_z), (eᵤ^eₓ^e_y, eᵤ^eₓ^e_z, eᵤ^e_y^e_z, eₓ^e_y^e_z), (eᵤ^eₓ^e_y^e_z,))

In [21]:
g4.e.obj

eᵤ^eₓ^e_y^e_z

In [22]:
g4.bases

((1,), (eᵤ, eₓ, e_y, e_z), (eᵤ*eₓ, eᵤ*e_y, eᵤ*e_z, eₓ*e_y, eₓ*e_z, e_y*e_z), (eᵤ*eₓ*e_y, eᵤ*eₓ*e_z, eᵤ*e_y*e_z, eₓ*e_y*e_z), (eᵤ*eₓ*e_y*e_z,))